In [ ]:
!pip install transformers

In [ ]:
!pip install newsapi-python

In [ ]:
#Usage
from newsapi import NewsApiClient
import pandas as pd
from transformers import pipeline

In [ ]:
# Init
newsapi = NewsApiClient(api_key='api_key')

In [ ]:
# List of tickers
tickers = ['NVDA', 'TSLA', 'MSFT', 'GOOG', 'META', 'MMM', 'BBY', 'JNJ', 'ORCL', 'MRK', 'VZ', 'BMY', 'PDD','AAPL', 'S&P 500']

# Initialize an empty list to store articles
all_articles_list = []

for ticker in tickers:
    all_articles = newsapi.get_everything(
        q=ticker,
        from_param='2023-09-01',
        to='2023-09-18',
        language='en',
        sort_by='relevancy',
        page=1
    )
    total_articles = all_articles['totalResults']
    article_counters[ticker] = total_articles
    articles = all_articles['articles']

    # Add a 'ticker' column with the current ticker for all articles
    for article in articles:
        article['ticker'] = ticker

    all_articles_list.extend(articles)

df = pd.DataFrame(all_articles_list)


# Load the sentiment analysis models
sentiment_analyzer = pipeline('sentiment-analysis')
sentiment_analyzer_finbert = pipeline(model="ProsusAI/finbert")
sentiment_analyzer_distilRoberta = pipeline(model="mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")

# Sentiment analysis 1 - distilbert-base-uncased-finetuned-sst-2-english
# Function to perform sentiment analysis
def get_sentiment(title):
    result = sentiment_analyzer(title)
    sentiment = result[0]['label']
    return sentiment

# Apply sentiment analysis to the 'title' column and create a new 'sentiment' column
df['sentiment'] = df['title'].apply(get_sentiment)

# Sentiment analysis 2 - finbert https://huggingface.co/ProsusAI/finbert
# Function to perform sentiment analysis
def get_sentiment_2(title):
    result = sentiment_analyzer_finbert(title)
    sentiment = result[0]['label']
    return sentiment

# Apply sentiment analysis to the 'title' column and create a new 'sentiment' column
df['sentiment_finbert'] = df['title'].apply(get_sentiment)


# Sentiment analysis 3 - distilRoberta-financial-sentiment - https://huggingface.co/mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis
def get_sentiment_3(title):
    result = sentiment_analyzer_distilRoberta(title)
    sentiment = result[0]['label']
    return sentiment

# Apply sentiment analysis to the 'title' column and create a new 'sentiment' column
df['sentiment_distilRoberta'] = df['title'].apply(get_sentiment)


# Function to determine majority sentiment
def majority_sentiment(row):
    sentiments = [row['sentiment_finbert'], row['sentiment_distilRoberta'], row['sentiment']]
    positive_count = sentiments.count('POSITIVE')
    negative_count = sentiments.count('NEGATIVE')
    if positive_count > negative_count:
        return 'POSITIVE'
    elif negative_count > positive_count:
        return 'NEGATIVE'
    else:
        return 'NEUTRAL'  # Handle ties as needed

# Create a new column 'majority_sentiment' based on majority sentiment
df['majority_sentiment'] = df.apply(majority_sentiment, axis=1)

df = df.drop(['sentiment_finbert', 'sentiment_distilRoberta', 'sentiment'], axis=1)